# Key Data Extraction App

## Intro
* We will create an app to **extract structured information from unstructured text**. Imagine, for example, that you want to extract the name, the lastname and the country of the users that submit comments in the website of your company.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-key-data-extraction.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-key-data-extraction**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Define what information you want to extract
* **We'll use Pydantic to define an schema to extract personal information**.
* Pydantic is a Python library used for data validation. It helps ensure that the data your program receives matches the format you expect, and it provides helpful error messages when the data doesn't conform to your specifications. Essentially, Pydantic allows you to enforce that the data structures in Python adhere to specific types and constraints, making your code more robust and error-resistant.
* **Document the attributes and the schema itself**: This information is sent to the LLM and is used to improve the quality of information extraction.
* Do not force the LLM to make up information! **We import Optional for the attributes allowing the LLM to output None if it doesn't know the answer**.
* When you use Optional in type hints, you are indicating that a variable can either be of a specified type or it can be None.

#### Let's define the data we want to extract from a person.
* See below that it is a good practice to write an explanatory doc-string (comments) to help the chat model understanding what data we want to extract.

In [6]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(
        default=None, description="The name of the person"
    )
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

## Define the Extractor

Our extractor will be a chain with the prompt template and a chat model with the extraction instructions.

In [7]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) You can introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

* We need to use a model that supports function/tool calling.
* Please review [the documentation](https://python.langchain.com/v0.2/docs/concepts/#function-tool-calling) for list of some models that can be used with this API.
* **We will use .with_structured_output() to add the extraction instructions to our chat model**.

In [8]:
chain = prompt | llm.with_structured_output(schema=Person)

## Try the extraction app
See how the app extracts the name, lastname and country form one user review:

In [9]:
comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, USA"

In [10]:
chain.invoke({"text": comment})

Person(name='Sarah', lastname='Johnson', country='USA')

* **Note that this extraction capability is generative**, which means that our model can perform a variety of tasks beyond the expected. For instance, the model could infer the gender of a user from their name, even when this information is not explicitly provided.

## Extraction of a list of entities rather than a single entity
* In real projects you will probably work with a large text that includes more than one user review. **We can extract the key data of several users by nesting Pydantic models**.
* See how the Data model definition is including the Person model. This is technically called "nesting" models.

In [11]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(
        default=None, description="The name of the person"
    )
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

See how now we are using the Data model with the llm:

In [12]:
chain = prompt | llm.with_structured_output(schema=Data)

In [13]:
comment = "I'm so impressed with this product! It has truly transformed how I approach my daily tasks. The quality exceeds my expectations, and the customer support is truly exceptional. I've already suggested it to all my colleagues and relatives. - Emily Clarke, Canada"

In [14]:
chain.invoke({"text": comment})

Data(people=[Person(name='Emily', lastname='Clarke', country='Canada')])

#### Let's now see this at work with a text with several reviews.

In [15]:
# Example input text that mentions multiple people
text_input = """
Alice Johnson from Canada recently reviewed a book she loved. Meanwhile, Bob Smith from the USA shared his insights on the same book in a different review. Both reviews were very insightful.
"""

# Invoke the processing chain on the text
response = chain.invoke({"text": text_input})

# Output the extracted data
response

Data(people=[Person(name='Alice', lastname='Johnson', country='Canada'), Person(name='Bob', lastname='Smith', country='USA')])

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-key-data-extraction.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-key-data-extraction.py